In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import sklearn
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('乱序所有数据.csv', encoding='utf-8_sig')
df = DataFrame(data)
df.head

<bound method NDFrame.head of          编号   直链淀粉含量g  直链淀粉比例     重量        A1        A2        A3        A4  \
0    234N-1  3.113456  23.785  13.09  0.408992  0.421020  0.417930  0.410035   
1    489N-1  2.626910  36.740   7.15  0.385900  0.398986  0.398872  0.391411   
2    483N-1  3.149256  34.760   9.06  0.426620  0.440990  0.438611  0.431118   
3    490N-1  4.424459  35.595  12.43  0.399484  0.404151  0.398467  0.397170   
4    232N-1  4.214154  37.660  11.19  0.379676  0.392528  0.389541  0.381101   
..      ...       ...     ...    ...       ...       ...       ...       ...   
102  020N-1  1.286309  15.295   8.41  0.519761  0.524981  0.533128  0.523232   
103  492N-1  2.224890  37.710   5.90  0.393300  0.405160  0.405411  0.397843   
104  017N-1  5.517554  33.460  16.49  0.461537  0.452760  0.460034  0.447231   
105  196N-1  4.266436  23.915  17.84  0.436723  0.429687  0.426973  0.422054   
106  101N-1  5.780725  37.295  15.50  0.357428  0.363206  0.367644  0.362402   

         

In [3]:
y = df.iloc[0:80, 2:3]
x= df.iloc[0:80, 3:]

y_validation = df.iloc[80:, 2:3]
x_validation = df.iloc[80:, 3:]

Y_total = df.iloc[:, 2:3]
X_total = df.iloc[:, 3:]

In [4]:
#归一化
scaler = MinMaxScaler().fit(x)
x = scaler.transform(x)

x_validation = scaler.transform(x_validation)
X_total = scaler.transform(X_total)

In [5]:
#降维
pca = PCA(n_components = 0.9999)
pca.fit(x)
x = pca.transform(x)

x_validation = pca.transform(x_validation)
print(pca.explained_variance_ratio_)

x.shape

[9.80016183e-01 9.69686964e-03 7.69707149e-03 1.94447771e-03
 2.44114484e-04 1.92358860e-04 5.89228359e-05 4.29805029e-05
 3.20981251e-05]


(80, 9)

In [6]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size= 0.25)
x_train.shape

(60, 9)

In [7]:
linear = linear_model.LinearRegression()
lasso = linear_model.Lasso(alpha=0.001, max_iter = 10000000)
ridge = linear_model.Ridge(alpha=0.001)  #----r2 0.75
huber = linear_model.HuberRegressor(max_iter=100000, epsilon=1.15)

In [8]:
model = ridge
model.fit(x_train, np.ravel(y_train))

y_train_predict = model.predict(x_train)
y_test_predict = model.predict(x_test)
y_validation_predict = model.predict(x_validation)    

In [9]:
r2_train_max = 0
r2_test_max = 0
r2_validation_max = 0
if r2_score(y_train, y_train_predict)>=r2_train_max and r2_score(y_test, y_test_predict)>=r2_test_max and r2_score(y_validation, y_validation_predict)>=r2_validation_max :
    
    r2_train_max = r2_score(y_train, y_train_predict)
    r2_test_max = r2_score(y_test, y_test_predict)
    r2_validation_max = r2_score(y_validation, y_validation_predict)    
    print("r2_train_max = {0},r2_test_max = {1}, r2_validation_max = {2}".format(r2_train_max,r2_test_max,r2_validation_max))

r2_train_max = 0.6012272113335739,r2_test_max = 0.18793924246578664, r2_validation_max = 0.7496512150608694


In [10]:
r2_score(y_train, y_train_predict)

0.6012272113335739

In [11]:
r2_score(y_test, y_test_predict)

0.18793924246578664

In [12]:

r2_score(y_validation, y_validation_predict)

0.7496512150608694

In [20]:
r2_train_max = 0
r2_test_max = 0
r2_validation_max = 0

for i in range(10):
    
#     df = sklearn.utils.shuffle(df)
    
    y = df.iloc[0:, 2:3]
    x= df.iloc[0:, 3:]
    #划分数据集
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size= 0.3)       
    
    #归一化
    scaler = MinMaxScaler().fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

#     #降维
    pca = PCA(n_components = 0.9999999)
    pca.fit(x_train)
    x_train = pca.transform(x_train)
    x_test = pca.transform(x_test)

    
    #模型选择
    model = huber
    model.fit(x_train, np.ravel(y_train))

    y_train_predict = model.predict(x_train)
    y_test_predict = model.predict(x_test)
#     y_validation_predict = model.predict(x_validation)   
    
    
    

    if r2_score(y_train, y_train_predict)>=r2_train_max and r2_score(y_test, y_test_predict)>=r2_test_max :
        r2_train_max = r2_score(y_train, y_train_predict)
        r2_test_max = r2_score(y_test, y_test_predict)
#         r2_validation_max = r2_score(y_validation, y_validation_predict)
        
    print(r2_score(y_train, y_train_predict) ,r2_score(y_test, y_test_predict))
    
print("r2_train_max = {0},r2_test_max = {1}, r2_validation_max = {2}".format(r2_train_max,r2_test_max,r2_validation_max))

0.7671659634043727 0.5321588206052024
0.7183510355735043 0.4212259557390532
0.7898516544675951 0.5437892275061142
0.798982493850058 0.4780196639464743
0.7539198442989539 0.5996470454906979
0.7686505060384834 0.6299142607705603
0.7630390494000849 0.6423459533236775
0.822303879938139 0.5087377602125394
0.7664357320656112 0.7274602568931949
0.7873535796777013 0.42314522964776424
r2_train_max = 0.7898516544675951,r2_test_max = 0.5437892275061142, r2_validation_max = 0


In [14]:
# kf = KFold(n_splits=5)
# for train_index, test_index in kf.split(x):
#     print('train_index', train_index, 'test_index', test_index)
#     train_x, train_y = x[train_index], y[train_index]
#     test_x, test_y = x[test_index], y[test_index]

In [15]:
# fig, ax = plt.subplots()
# ax.scatter(y, predicted, c="blue", edgecolors="red")
# ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k', lw=2)
# ax.set_xlabel('y')
# ax.set_ylabel('prediction')
# plt.show()